# Lemon Agent | レモンエージェント

> [Lemon Agent](https://github.com/felixbrock/lemon-agent)は、`Airtable`、`Hubspot`、`Discord`、`Notion`、`Slack`、`Github`などのツールで正確で信頼性の高い読み書き操作を可能にし、数分で強力なAIアシスタントを構築し、ワークフローを自動化するのに役立ちます。
>
> > [Lemon Agent](https://github.com/felixbrock/lemon-agent) helps you build powerful AI assistants in minutes and automate workflows by allowing for accurate and reliable read and write operations in tools like `Airtable`, `Hubspot`, `Discord`, `Notion`, `Slack` and `Github`.

[完全なドキュメントはこちら](https://github.com/felixbrock/lemonai-py-client)でご確認いただけます。

> See [full docs here](https://github.com/felixbrock/lemonai-py-client).

現在利用可能なコネクタの多くは読み取り専用の操作に特化しており、LLMの可能性を制限しています。一方で、エージェントは時折、コンテキストや指示が不足していることにより、誤情報を生み出す傾向があります。

> Most connectors available today are focused on read-only operations, limiting the potential of LLMs. Agents, on the other hand, have a tendency to hallucinate from time to time due to missing context or instructions.

`Lemon AI`を使用すると、エージェントに信頼性の高い読み書き操作を行うための明確に定義されたAPIへのアクセスを提供することが可能です。さらに、`Lemon AI`の機能により、不確実性の際にモデルが頼りにできるワークフローを静的に定義する方法を提供することで、幻覚のリスクをさらに低減できます。

> With `Lemon AI`, it is possible to give your agents access to well-defined APIs for reliable read and write operations. In addition, `Lemon AI` functions allow you to further reduce the risk of hallucinations by providing a way to statically define workflows that the model can rely on in case of uncertainty.



## Quick Start | クイックスタート

以下のクイックスタートでは、内部ツールとの対話を含むワークフローを自動化するために、Lemon AIとエージェントを組み合わせて使用する方法を実演しています。

> The following quick start demonstrates how to use Lemon AI in combination with Agents to automate workflows that involve interaction with internal tooling.



### 1. Install Lemon AI | 1. Lemon AIをインストールする

Python 3.8.1以上が必要です。

> Requires Python 3.8.1 and above.

PythonプロジェクトでLemon AIを使用するには、`pip install lemonai`を実行してください

> To use Lemon AI in your Python project run `pip install lemonai`

これにより、対応するLemon AIクライアントがインストールされ、その後スクリプトにインポートできます。

> This will install the corresponding Lemon AI client which you can then import into your script.

このツールは、Pythonパッケージのlangchainとloguruを使用しています。Lemon AIのインストールでエラーが発生した場合は、まず両方のパッケージをインストールしてから、Lemon AIパッケージをインストールしてください。

> The tool uses Python packages langchain and loguru. In case of any installation errors with Lemon AI, install both packages first and then install the Lemon AI package.



### 2. Launch the Server | 2. サーバーを起動する

Lemon AIが提供するすべてのツールとあなたのエージェントのインタラクションは、[Lemon AI Server](https://github.com/felixbrock/lemonai-server)によって管理されます。Lemon AIを使用するためには、ローカルマシンでサーバーを実行して、Lemon AI Pythonクライアントが接続できるようにする必要があります。

> The interaction of your agents and all tools provided by Lemon AI is handled by the [Lemon AI Server](https://github.com/felixbrock/lemonai-server). To use Lemon AI you need to run the server on your local machine so the Lemon AI Python client can connect to it.



### 3. Use Lemon AI with Langchain | 3. LangchainでLemon AIを使用する



Lemon AIは、関連するツールの適切な組み合わせを見つけ出すか、あるいはLemon AI Functionsを代わりに使用することで、与えられたタスクを自動的に解決します。以下の例では、Hackernewsからユーザー情報を取得し、それをAirtableのテーブルに記録する方法を示しています：

> Lemon AI automatically solves given tasks by finding the right combination of relevant tools or uses Lemon AI Functions as an alternative. The following example demonstrates how to retrieve a user from Hackernews and write it to a table in Airtable:



#### (Optional) Define your Lemon AI Functions | （オプション）Lemon AI関数を定義してください



OpenAIの機能に似て、Lemon AIではワークフローを再利用可能な関数として定義するオプションを提供しています。これらの関数は、できる限り決定論的な振る舞いに近づけることが特に重要なユースケースにおいて定義できます。特定のワークフローは、別のlemonai.jsonファイル内で定義することができます：

> Similar to [OpenAI functions](https://openai.com/blog/function-calling-and-other-api-updates), Lemon AI provides the option to define workflows as reusable functions. These functions can be defined for use cases where it is especially important to move as close as possible to near-deterministic behavior. Specific workflows can be defined in a separate lemonai.json:



```json
[
  {
    "name": "Hackernews Airtable User Workflow",
    "description": "retrieves user data from Hackernews and appends it to a table in Airtable",
    "tools": ["hackernews-get-user", "airtable-append-data"]
  }
]
```



モデルはこれらの機能にアクセスでき、与えられたタスクを解決する際に自己選択ツールよりもこれらの機能を優先して使用します。あなたがする必要があるのは、プロンプトにその機能名を含めることで、エージェントにその機能を使用するよう指示することです。

> Your model will have access to these functions and will prefer them over self-selecting tools to solve a given task. All you have to do is to let the agent know that it should use a given function by including the function name in the prompt.



#### Include Lemon AI in your Langchain project | LangchainプロジェクトにLemon AIを組み込んでください



In [1]:
import os

from langchain.llms import OpenAI
from lemonai import execute_workflow

#### Load API Keys and Access Tokens | APIキーとアクセストークンを読み込む

認証が必要なツールを使用するには、対応するアクセス資格情報を「{ツール名}\_{認証文字列}」の形式で環境に保存する必要があります。認証文字列は、APIキーの場合は\["API\_KEY", "SECRET\_KEY", "SUBSCRIPTION\_KEY", "ACCESS\_KEY"]のいずれか、認証トークンの場合は\["ACCESS\_TOKEN", "SECRET\_TOKEN"]のいずれかになります。例えば、「OPENAI\_API\_KEY」、「BING\_SUBSCRIPTION\_KEY」、「AIRTABLE\_ACCESS\_TOKEN」などがあります。

> To use tools that require authentication, you have to store the corresponding access credentials in your environment in the format "{tool name}\_{authentication string}" where the authentication string is one of \["API\_KEY", "SECRET\_KEY", "SUBSCRIPTION\_KEY", "ACCESS\_KEY"] for API keys or \["ACCESS\_TOKEN", "SECRET\_TOKEN"] for authentication tokens. Examples are "OPENAI\_API\_KEY", "BING\_SUBSCRIPTION\_KEY", "AIRTABLE\_ACCESS\_TOKEN".



In [2]:
""" Load all relevant API Keys and Access Tokens into your environment variables """
os.environ["OPENAI_API_KEY"] = "*INSERT OPENAI API KEY HERE*"
os.environ["AIRTABLE_ACCESS_TOKEN"] = "*INSERT AIRTABLE TOKEN HERE*"

In [ ]:
hackernews_username = "*INSERT HACKERNEWS USERNAME HERE*"
airtable_base_id = "*INSERT BASE ID HERE*"
airtable_table_id = "*INSERT TABLE ID HERE*"

""" Define your instruction to be given to your LLM """
prompt = f"""Read information from Hackernews for user {hackernews_username} and then write the results to
Airtable (baseId: {airtable_base_id}, tableId: {airtable_table_id}). Only write the fields "username", "karma"
and "created_at_i". Please make sure that Airtable does NOT automatically convert the field types.
"""

"""
Use the Lemon AI execute_workflow wrapper 
to run your Langchain agent in combination with Lemon AI  
"""
model = OpenAI(temperature=0)

execute_workflow(llm=model, prompt_string=prompt)

### 4. Gain transparency on your Agent's decision making | 4. エージェントの意思決定の透明性を高める

エージェントが与えられたタスクを解決するためにLemon AIツールとどのように対話しているかを透明にするため、行われた全ての決定、使用されたツール、および実行された操作はローカルの`lemonai.log`ファイルに記録されます。LLMエージェントがLemon AIツール群と対話する際には、それに対応するログエントリが作成されます。

> To gain transparency on how your Agent interacts with Lemon AI tools to solve a given task, all decisions made, tools used and operations performed are written to a local `lemonai.log` file. Every time your LLM agent is interacting with the Lemon AI tool stack a corresponding log entry is created.

```log
2023-06-26T11:50:27.708785+0100 - b5f91c59-8487-45c2-800a-156eac0c7dae - hackernews-get-user
2023-06-26T11:50:39.624035+0100 - b5f91c59-8487-45c2-800a-156eac0c7dae - airtable-append-data
2023-06-26T11:58:32.925228+0100 - 5efe603c-9898-4143-b99a-55b50007ed9d - hackernews-get-user
2023-06-26T11:58:43.988788+0100 - 5efe603c-9898-4143-b99a-55b50007ed9d - airtable-append-data
```

[Lemon AI Analytics](https://github.com/felixbrock/lemon-agent/blob/main/apps/analytics/README.md)を利用することで、どのツールがどの順序でどれほど頻繁に使用されているかを容易に把握できます。その結果、エージェントの意思決定能力における弱点を見つけ出し、Lemon AIの関数を定義することで、より決定論的な行動へと移行することが可能になります。

> By using the [Lemon AI Analytics](https://github.com/felixbrock/lemon-agent/blob/main/apps/analytics/README.md) you can easily gain a better understanding of how frequently and in which order tools are used. As a result, you can identify weak spots in your agent’s decision-making capabilities and move to a more deterministic behavior by defining Lemon AI functions.

